<a href="https://colab.research.google.com/github/2303A52337/Explainable-AI-Lab-Practical/blob/main/ex_ai_lab_exam_1_2337.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==================== SETUP ====================
!pip install opencv-python kagglehub tqdm tensorflow -q

# ==================== IMPORTS ====================
import kagglehub
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

# ==================== DOWNLOAD DATASET ====================
path = kagglehub.dataset_download("piyushraj25/farming")
print("✅ Dataset Path:", path)

# ==================== LOAD IMAGES ====================
img_size = (128, 128)
images, labels = [], []

for file in tqdm(os.listdir(path)):
    if file.lower().endswith('.tif'):
        label = file.split('_')[0]  # e.g. "NDVI_..." → NDVI
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)
            img = img.astype('float32') / 255.0
            img = np.nan_to_num(img, nan=0.0)  # remove NaN pixels
            images.append(img)
            labels.append(label)

images = np.array(images)
labels = np.array(labels)

print("📸 Total Images:", len(images))
print("🏷️ Classes:", np.unique(labels))

# ==================== ENCODE + SPLIT ====================
le = LabelEncoder()
y = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2, random_state=42)

# ==================== 1️⃣ MACHINE LEARNING (SVM) ====================
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# Replace NaNs/Infs before scaling
X_train_flat = np.nan_to_num(X_train_flat, nan=0.0, posinf=0.0, neginf=0.0)
X_test_flat = np.nan_to_num(X_test_flat, nan=0.0, posinf=0.0, neginf=0.0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"\n🌾 SVM (Machine Learning) Accuracy: {svm_acc:.3f}")

# ==================== 2️⃣ DEEP LEARNING (CNN) ====================
cnn = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size + (3,)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')
])

cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), verbose=1)

loss, cnn_acc = cnn.evaluate(X_test, y_test, verbose=0)
print(f"🤖 CNN (Deep Learning) Accuracy: {cnn_acc:.3f}")

# ==================== RESULT SUMMARY ====================
print("\n================= RESULTS =================")
print(f"SVM Accuracy : {svm_acc*100:.2f}%")
print(f"CNN Accuracy : {cnn_acc*100:.2f}%")
print("===========================================")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 736.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 139.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Using Colab cache for faster access to the 'farming' dataset.
✅ Dataset Path: /kaggle/input/farming


100%|██████████| 356/356 [00:04<00:00, 81.48it/s] 


📸 Total Images: 356
🏷️ Classes: ['FalseColor' 'NDSI' 'NDVI' 'NDWI' 'SWIR' 'TCI']

🌾 SVM (Machine Learning) Accuracy: 0.569


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 299ms/step - accuracy: 0.3485 - loss: 8.4673 - val_accuracy: 0.2500 - val_loss: 2.5153
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.3302 - loss: 2.0527 - val_accuracy: 0.3750 - val_loss: 1.3435
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 277ms/step - accuracy: 0.5163 - loss: 1.1627 - val_accuracy: 0.4306 - val_loss: 0.9868
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 277ms/step - accuracy: 0.5492 - loss: 0.8409 - val_accuracy: 0.4167 - val_loss: 0.9891
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 275ms/step - accuracy: 0.4586 - loss: 0.9058 - val_accuracy: 0.4444 - val_loss: 1.0059
🤖 CNN (Deep Learning) Accuracy: 0.444

================= RESULTS =================
SVM Accuracy : 56.94%
CNN Accuracy : 44.44%
